In [1]:
import pandas as pd

In [56]:
pip install ipwhois

     ---------------------------------------- 0.0/73.5 kB ? eta -:--:--
     ---------------------- ----------------- 41.0/73.5 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 73.5/73.5 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/208.3 kB ? eta -:--:--
     ------------------------------------- 208.3/208.3 kB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\ujn9\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
keywords = pd.read_csv('./dataset/금지어keyword.csv')
blacklist = pd.read_csv('./dataset/블랙리스트정보v2 - 복사본.csv', encoding='latin1')
normal_item_2022 = pd.read_csv('./dataset/정상상품.csv')
weird_item_2022 = pd.read_csv('./dataset/이상상품.csv')
normal_member = pd.read_csv('./dataset/정상회원데이터.csv')
weird_member = pd.read_csv('./dataset/이상회원데이터.csv')
inquir = pd.read_csv('./dataset/인콰이어리데이터.csv')

C:\Users\ujn9\AppData\Local\Temp\ipykernel_5584\3917221033.py:5: DtypeWarning: Columns (10,12,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  normal_member = pd.read_csv('./dataset/정상회원데이터.csv')
C:\Users\ujn9\AppData\Local\Temp\ipykernel_5584\3917221033.py:6: DtypeWarning: Columns (7,10,11,12,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  weird_member = pd.read_csv('./dataset/이상회원데이터.csv')


In [8]:
import sklearn

## 불필요컬럼 제거

In [18]:
normal_item_2022['SHOW_CHK'].value_counts()
weird_item_2022['SHOW_CHK'].value_counts()

SHOW_CHK
1    94243
0    67655
2    28118
3    22577
Name: count, dtype: int64

In [24]:
normal_item_2022['SHOW_CHK'].value_counts()

SHOW_CHK
0    538722
Name: count, dtype: int64

In [17]:
weird_item_2022.head()

,MEMBER_ID,GCATALOG_ID,CATALOG_ID,CATALOG_NM,CATEGORYM_ID,KEYWORD,DISPLAY,CATALOG_DESC,INPUT_DT,UPDATE_DT,REMOTEIP,SHOW_CHK
0,shangyuanshibang,GC11736713,CA11736718,1-BOC-4-(4-BROMO-PHENYLAMINO)-PIPERIDINECas 44...,212815,1-BOC-4- 4-BROMO-PHENYLAM,0,"<div class=""c_lis_topt1"" id=""m1"" style=""paddin...",20230228100427,20230228100427,219.78.227.175,2
1,shangyuanshibang,GC11736706,CA11736712,High Purity N- (Tert-Butoxycarbonyl) -4-Piperi...,212815,N- tert-Butoxycarbonyl -4,0,"<div class=""c_lis_topt1"" id=""m1"" style=""paddin...",20230228094237,20230228094237,42.3.79.92,2
2,shangyuanshibang,GC11736708,CA11736711,"1,1-dimethylethyl Ester CAS 288573-56-8",212815,1-BOC-4- 4-FLUORO-PHENYLA,0,"<div class=""c_lis_topt1"" id=""m1"" style=""paddin...",20230228093820,20230228093820,42.3.79.92,2
3,shangyuanshibang,GC11736704,CA11736705,White Crystal N-Isopropylbenzylamine CAS 102-9...,212815,N-Isopropylbenzylamine,0,"<div class=""c_lis_topt1"" id=""m1"" style=""paddin...",20230228091028,20230228091028,42.3.79.92,2
4,5527,GC11736699,CA11736700,Soybean GMO & NON-GMO,11607,Soybean soybean gmo,0,"<ol><li class=""six-page-style-el"">Soja GMO &am...",20230228075935,20230228075935,131.196.216.214,2


## DISPLAY 컬럼 Y=0 , N=1로 치환
## SHOW_CHK 컬럼 S=0, P=1, R=2, B=3 치환

In [12]:
normal_item_2022['DISPLAY'] = normal_item_2022['DISPLAY'].replace({'Y': 0, 'N': 1})
weird_item_2022['DISPLAY'] = weird_item_2022['DISPLAY'].replace({'Y': 0, 'N': 1})

In [23]:
weird_item_2022['SHOW_CHK'] = weird_item_2022['SHOW_CHK'].replace({'S': 0, 'P': 1, 'R': 2, 'B': 3})
normal_item_2022['SHOW_CHK'] = normal_item_2022['SHOW_CHK'].replace({'A': 0})

In [26]:
columns_to_keep = ['CATALOG_NM', 'KEYWORD', 'DISPLAY', 'CATALOG_DESC', 'INPUT_DT', 'UPDATE_DT', 'REMOTEIP', 'SHOW_CHK']
normal_item_2022 = normal_item_2022.loc[:, columns_to_keep]

In [27]:
columns_to_keep = ['CATALOG_NM', 'KEYWORD', 'DISPLAY', 'INPUT_DT', 'UPDATE_DT', 'REMOTEIP', 'SHOW_CHK']
normal_item_2022 = normal_item_2022.loc[:, columns_to_keep]

In [28]:
normal_item_2022.columns

Index(['CATALOG_NM', 'KEYWORD', 'DISPLAY', 'INPUT_DT', 'UPDATE_DT', 'REMOTEIP',
       'SHOW_CHK'],
      dtype='object')

In [29]:
columns_to_keep = ['CATALOG_NM', 'KEYWORD', 'DISPLAY', 'INPUT_DT', 'UPDATE_DT', 'REMOTEIP', 'SHOW_CHK']
weird_item_2022 = weird_item_2022.loc[:, columns_to_keep]

## 이상상품 판단 컬럼 추가

In [30]:
normal_item_2022['judge'] = 1
weird_item_2022['judge'] = 0

In [43]:
weird_item_2022['DISPLAY'].value_counts()

DISPLAY
0    189591
1     23002
Name: count, dtype: int64

In [44]:
normal_item_2022['DISPLAY'].value_counts()

DISPLAY
     470030
n     68692
Name: count, dtype: int64

In [31]:
import nltk

In [32]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ujn9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

## 데이터 클렌징

### 소문자 변경, Stopwords 제거, 어근추출

In [34]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Convert to lowercase
        text = text.lower()

        # Remove special characters
        text = ''.join([char for char in text if char not in string.punctuation])

    return text

def remove_stopwords(text):
    if isinstance(text, str):  # Check if the value is a string
        stop_words = set(stopwords.words('english'))
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)
    return text

def apply_stemming(text):
    if isinstance(text, str):  # Check if the value is a string
        stemmer = PorterStemmer()
        words = text.split()
        stemmed_words = [stemmer.stem(word) for word in words]
        return ' '.join(stemmed_words)
    return text

# Apply text processing to specified columns
columns_to_process = ['DISPLAY', 'SHOW_CHK','CATALOG_NM', 'KEYWORD']
for column in columns_to_process:
    normal_item_2022[column] = normal_item_2022[column].apply(preprocess_text)
    normal_item_2022[column] = normal_item_2022[column].apply(remove_stopwords)
    normal_item_2022[column] = normal_item_2022[column].apply(apply_stemming)
    weird_item_2022[column] = weird_item_2022[column].apply(preprocess_text)
    weird_item_2022[column] = weird_item_2022[column].apply(remove_stopwords)
    weird_item_2022[column] = weird_item_2022[column].apply(apply_stemming)  # or apply_lemmatization if you prefer

# Displaying the updated DataFrame
normal_item_2022.head()

,CATALOG_NM,KEYWORD,DISPLAY,INPUT_DT,UPDATE_DT,REMOTEIP,SHOW_CHK,judge
0,new season 2023 black tea ctc milk tea fulmex ...,milk teactc black teafulmex tea,,20230228005732,20230228005732,14.177.113.144,0,1
1,cow mat,rubber matcow matrubb floor matpashu chataidai...,,20230227231604,20230227231604,49.34.172.73,0,1
2,5 1 multifunct 4d hifu ultra lift needl rf vma...,4d hifu ultra lift5 1 hifu4d hifu lift4d needl...,,20230227224740,20230227224740,159.138.61.247,0,1
3,6mm 612 electr dc mini vibrat motor vr massag ...,vibrat motordc motorcoreless motormini motorsm...,,20230227223506,20230227223506,103.167.134.61,0,1
4,pe foam sheet extrud epe foam sheet extrud foa...,pe foam sheet extruderp foam extrusionep foam ...,,20230227214151,20230227214151,118.41.162.204,0,1


In [45]:
merged_df['SHOW_CHK'].value_counts()

SHOW_CHK
0    606377
1     94243
2     28118
3     22577
Name: count, dtype: int64

In [46]:
merged_df['DISPLAY'].value_counts()

DISPLAY
     470030
0    189591
n     68692
1     23002
Name: count, dtype: int64

In [51]:
merged_df.drop(columns=['DISPLAY'], inplace=True)

## 이상상품, 정상상품 CONCAT

In [49]:
merged_df = pd.concat([normal_item_2022, weird_item_2022], ignore_index=True)

# Display the merged DataFrame
merged_df['judge'].value_counts()

judge
1    538722
0    212593
Name: count, dtype: int64

In [53]:
merged_df['DISPLAY'].value_counts()

KeyError: 'DISPLAY'

In [52]:
merged_df.columns

Index(['CATALOG_NM', 'KEYWORD', 'INPUT_DT', 'UPDATE_DT', 'REMOTEIP',
       'SHOW_CHK', 'judge'],
      dtype='object')

## REMOTEIP 컬럼 클렌징 및 국가코드 컬럼추가

In [57]:
from ipwhois import IPWhois

# Assuming 'merged_df' is your DataFrame
# If you have not created the DataFrame yet, make sure to do that before running this code

# Function to get country code from IP address
def get_country_code(ip):
    try:
        obj = IPWhois(ip)
        results = obj.lookup_rdap()
        return results['asn_country_code']
    except Exception as e:
        print(f"Error for IP {ip}: {e}")
        return None

# Create a new column without periods
merged_df['REMOTEIP_no_period'] = merged_df['REMOTEIP'].str.replace('.', '')

# Apply the function to get country codes
merged_df['COUNTRY_CODE'] = merged_df['REMOTEIP'].apply(get_country_code)

# Display the updated DataFrame
print(merged_df)

Error for IP 35.214.59.141: ASN lookup failed with no more methods to try.
Error for IP 173.245.203.231: Parsing failed for "" with exception: list index out of range.
Error for IP 89.187.162.107: ASN lookup failed with no more methods to try.
Error for IP 23.154.81.87: ASN lookup failed with no more methods to try.


##  NA값 채우기

In [ ]:
from datetime import datetime, timedelta

# Assuming 'merged_df' is your DataFrame
# If you have not created the DataFrame yet, make sure to do that before running this code

# Check missing values in all columns
missing_values = merged_df.isnull().sum()
print("Missing Values in Each Column:")
print(missing_values)

# Fill missing values in 'CATALOG_NM' and 'KEYWORD' columns with 0
merged_df[['CATALOG_NM', 'KEYWORD']] = merged_df[['CATALOG_NM', 'KEYWORD']].fillna('0')

# Fill missing values in 'INPUT_DT' and 'UPDATE_DT' columns with dates 10 days apart
# if either of the two columns has a missing value
for index, row in merged_df.iterrows():
    if pd.isnull(row['INPUT_DT']):
        row['INPUT_DT'] = datetime.now() - timedelta(days=10)
    elif pd.isnull(row['UPDATE_DT']):
        row['UPDATE_DT'] = datetime.now() + timedelta(days=10)

# Fill missing values in other columns with 0
merged_df.fillna(0, inplace=True)

# Display the updated DataFrame
print("\nUpdated DataFrame:")
print(merged_df)

## 카탈로그, 키워드 열 카운트벡터

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
merged_df.head()

,CATALOG_NM,KEYWORD,INPUT_DT,UPDATE_DT,REMOTEIP,SHOW_CHK,judge
0,new season 2023 black tea ctc milk tea fulmex ...,milk teactc black teafulmex tea,20230228005732,20230228005732,14.177.113.144,0,1
1,cow mat,rubber matcow matrubb floor matpashu chataidai...,20230227231604,20230227231604,49.34.172.73,0,1
2,5 1 multifunct 4d hifu ultra lift needl rf vma...,4d hifu ultra lift5 1 hifu4d hifu lift4d needl...,20230227224740,20230227224740,159.138.61.247,0,1
3,6mm 612 electr dc mini vibrat motor vr massag ...,vibrat motordc motorcoreless motormini motorsm...,20230227223506,20230227223506,103.167.134.61,0,1
4,pe foam sheet extrud epe foam sheet extrud foa...,pe foam sheet extruderp foam extrusionep foam ...,20230227214151,20230227214151,118.41.162.204,0,1


## 라벨 인코딩

In [30]:
from sklearn.preprocessing import LabelEncoder

In [31]:
if 'CATALOG_NM' in merged_df.columns and 'KEYWORD' in merged_df.columns:
    label_encoder = LabelEncoder()

    # Label encode 'CATALOG_NM' and 'KEYWORD' columns
    merged_df['CATALOG_NM_encoded'] = label_encoder.fit_transform(merged_df['CATALOG_NM'])
    merged_df['KEYWORD_encoded'] = label_encoder.fit_transform(merged_df['KEYWORD'])

    # Drop the original 'CATALOG_NM' and 'KEYWORD' columns
    merged_df = merged_df.drop(['CATALOG_NM', 'KEYWORD'], axis=1)

    # Display the updated DataFrame
    print(merged_df)
else:
    print("The necessary columns 'CATALOG_NM' and/or 'KEYWORD' are missing.")

              INPUT_DT       UPDATE_DT     REMOTEIP  judge  \
0       20230228005732  20230228005732  14177113144      1   
1       20230227231604  20230227231604    493417273      1   
2       20230227224740  20230227224740  15913861247      1   
3       20230227223506  20230227223506  10316713461      1   
4       20230227214151  20230227214151  11841162204      1   
...                ...             ...          ...    ...   
751310  20180101031323  20180101031323   9122621280      0   
751311  20180101031009  20180101031009   9122621280      0   
751312  20180101030646  20180101030646   9122621280      0   
751313  20180101030314  20180101030314   9122621280      0   
751314  20180101025359  20180101025359   9122621280      0   

        CATALOG_NM_encoded  KEYWORD_encoded  
0                   309996           213049  
1                   135858           281551  
2                    33402             8042  
3                    38098           341147  
4                   33549

In [55]:
merged_df.head()

,CATALOG_NM,KEYWORD,INPUT_DT,UPDATE_DT,REMOTEIP,SHOW_CHK,judge
0,new season 2023 black tea ctc milk tea fulmex ...,milk teactc black teafulmex tea,20230228005732,20230228005732,14.177.113.144,0,1
1,cow mat,rubber matcow matrubb floor matpashu chataidai...,20230227231604,20230227231604,49.34.172.73,0,1
2,5 1 multifunct 4d hifu ultra lift needl rf vma...,4d hifu ultra lift5 1 hifu4d hifu lift4d needl...,20230227224740,20230227224740,159.138.61.247,0,1
3,6mm 612 electr dc mini vibrat motor vr massag ...,vibrat motordc motorcoreless motormini motorsm...,20230227223506,20230227223506,103.167.134.61,0,1
4,pe foam sheet extrud epe foam sheet extrud foa...,pe foam sheet extruderp foam extrusionep foam ...,20230227214151,20230227214151,118.41.162.204,0,1


In [33]:
merged_df['judge'].value_counts()

judge
1    538722
0    212593
Name: count, dtype: int64

## 원핫인코딩

In [ ]:
if 'CATALOG_NM' in merged_df.columns and 'KEYWORD' in merged_df.columns:
    # One-hot encode 'CATALOG_NM' and 'KEYWORD' columns
    merged_df = pd.get_dummies(merged_df, columns=['CATALOG_NM', 'KEYWORD'], drop_first=True)

    # Display the updated DataFrame
    print(merged_df)
else:
    print("The necessary columns 'CATALOG_NM' and/or 'KEYWORD' are missing.")

## 머신러닝 모델 패키지 설치

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 디시젼 트리, 랜덤 포레스트 모델

In [ ]:
X = merged_df.drop('judge', axis=1)
y = merged_df['judge']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Model
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Random Forest Model
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_decision_tree = decision_tree_model.predict(X_test)
y_pred_random_forest = random_forest_model.predict(X_test)

# Evaluate Decision Tree Model
print("Decision Tree Model:")
print("Accuracy:", accuracy_score(y_test, y_pred_decision_tree))
print("Classification Report:\n", classification_report(y_test, y_pred_decision_tree))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_decision_tree))

# Evaluate Random Forest Model
print("\nRandom Forest Model:")
print("Accuracy:", accuracy_score(y_test, y_pred_random_forest))
print("Classification Report:\n", classification_report(y_test, y_pred_random_forest))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_random_forest))

Decision Tree Model:
Accuracy: 0.9141438677518751
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.85      0.85     42437
           1       0.94      0.94      0.94    107826

    accuracy                           0.91    150263
   macro avg       0.89      0.90      0.89    150263
weighted avg       0.91      0.91      0.91    150263

Confusion Matrix:
 [[ 36194   6243]
 [  6658 101168]]

Random Forest Model:
Accuracy: 0.9411365405988168
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.83      0.89     42437
           1       0.94      0.98      0.96    107826

    accuracy                           0.94    150263
   macro avg       0.94      0.91      0.92    150263
weighted avg       0.94      0.94      0.94    150263

Confusion Matrix:
 [[ 35355   7082]
 [  1763 106063]]


## 실제 데이터 입력

### 실제 데이터에서 랜덤으로 20개 추출 

In [ ]:
random_sample_df = merged_df.sample(n=20, random_state=42)
random_sample_df.head(20)

,INPUT_DT,UPDATE_DT,REMOTEIP,judge,CATALOG_NM_encoded,KEYWORD_encoded
424815,20181010135044,20181010135044,183207172240,1,347211,99779
538734,20230227170552,20230227170552,22024612462,0,88588,46017
303681,20191210143447,20191230152344,149129120204,1,24908,233063
598332,20220607144934,202206080908,10413131150,0,399644,303949
319122,20190924150655,20220808170338,596169206,1,117572,167642
432512,20180910155346,20180910155346,11388198174,1,26898,5620
601842,20220524000824,20220524000824,462463200,0,297158,220089
349133,20190602131327,201907190921,91132137102,1,450841,235447
727159,20190110183715,202107241449,3757116120,0,45350,348408
274378,20200422153242,20200422153242,22024221130,1,87510,101679


In [ ]:
random_sample_df = random_sample_df.drop('judge', axis=1)

### 모델 성능 결과

In [ ]:
new_predictions = random_forest_model.predict(random_sample_df)

# Display the predictions
print("Predictions for new data:", new_predictions)

Predictions for new data: [1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1]


In [ ]:
Predictions for new data: [1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1]
Result for new data:      [1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1]